In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.style.use('science')

In [2]:
df = pd.read_csv("../data/results-benchmark.csv")

In [3]:
df.head()

,n,idx,distribution,travel,serv,cost_profile,dotsp,enum,lns,mtsp,...,svf,tsp,dotsp_gap,tsp_gap,svf_gap,mtsp_gap,lns_gap,nnsvf_gap,enum_gap,best
0,6,0,0,0,0,"(0.5, 2.5, 10)",416.0,395.078292,395.078,447.0,...,473.0,416.0,5.295663,5.295663,19.723194,13.142215,0.000000,7.067465,0.019821,395.078000
1,6,0,0,0,0,"(1.0, 2.5, 10)",481.0,460.767292,460.767,512.0,...,573.0,481.0,4.391156,4.391156,24.357864,11.119069,0.000000,8.731745,0.166803,460.767000
2,6,0,0,0,0,"(2.0, 2.5, 10)",610.0,592.145292,592.145,644.0,...,773.0,610.0,3.015309,3.015309,30.542350,8.757146,0.000000,11.121431,0.024543,592.145000
3,6,0,0,0,1,"(0.5, 2.5, 10)",683.0,621.380942,621.381,734.0,...,694.0,683.0,9.916471,9.916471,11.686721,18.123996,0.000009,10.560198,0.061343,621.380942
4,6,0,0,0,1,"(1.0, 2.5, 10)",748.0,691.553479,692.968,805.0,...,794.0,748.0,8.162279,8.162279,14.813969,16.404591,0.204543,12.934144,0.000000,691.553479


## Preprocess data

In [4]:
# Get the weight coefficient for the travel times
df['$\omega^T$'] = df.cost_profile.str[1:4]
df['$n$'] = df['n']

# Create new gap columns with algorithm names in uppercase 
for col in [col for col in df.columns if col.endswith("_gap")]:
    alg = col.removesuffix("_gap").upper() # algorithm name in uppercase
    df[alg] = df[col]
    
ALGS = ["LNS", "MTSP", "DOTSP", "NNSVF"]    

## Small instances $(n \leq 10)$

In [5]:
res = df[df.n <= 10]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$'])[ALGS].mean().round(2)

print(res[ALGS].to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{llrrrr}
\toprule
 &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{6} & 0.5 & 1.69 & 8.33 & 5.67 & 11.08 \\
 & 1.0 & 1.35 & 6.48 & 4.54 & 12.93 \\
 & 2.0 & 1.38 & 4.97 & 3.32 & 16.12 \\
\cline{1-6}
\multirow[t]{3}{*}{8} & 0.5 & 2.52 & 7.80 & 5.51 & 9.58 \\
 & 1.0 & 1.84 & 6.19 & 4.28 & 11.78 \\
 & 2.0 & 1.40 & 5.18 & 2.96 & 15.69 \\
\cline{1-6}
\multirow[t]{3}{*}{10} & 0.5 & 2.68 & 9.10 & 7.07 & 10.71 \\
 & 1.0 & 2.07 & 7.21 & 5.46 & 13.68 \\
 & 2.0 & 1.29 & 5.23 & 3.71 & 19.15 \\
\cline{1-6}
\bottomrule
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
 & LNS & MTSP & DOTSP & NNSVF \\
\midrule
0 & 1.80 & 6.72 & 4.72 & 13.41 \\
\bottomrule
\end{tabular}



LNS  MTSP  DOTSP  NNSVF
$n$ $\omega^T$                          
6   0.5         1.69  8.33   5.67  11.08
    1.0         1.35  6.48   4.54  12.93
    2.0         1.38  4.97   3.32  16.12
8   0.5         2.52  7.80   5.51   9.58
    1.0         1.84  6.19   4.28  11.78
    2.0         1.40  5.18   2.96  15.69
10  0.5         2.68  9.10   7.07  10.71
    1.0         2.07  7.21   5.46  13.68
    2.0         1.29  5.23   3.71  19.15

In [6]:
res.mean()

LNS       1.802222
MTSP      6.721111
DOTSP     4.724444
NNSVF    13.413333
dtype: float64

In [7]:
res = df[df.n <= 10]

# Get relevant columns and groupby
res = res.groupby(['$n$', 'serv'])[ALGS].mean().round(2)

print(res[ALGS].to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{llrrrr}
\toprule
 &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & serv &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{6} & 0 & 0.77 & 4.41 & 3.46 & 13.61 \\
 & 1 & 2.17 & 8.78 & 5.56 & 13.15 \\
\cline{1-6}
\multirow[t]{2}{*}{8} & 0 & 0.84 & 4.57 & 2.86 & 12.69 \\
 & 1 & 3.00 & 8.20 & 5.64 & 12.01 \\
\cline{1-6}
\multirow[t]{2}{*}{10} & 0 & 1.35 & 6.07 & 4.13 & 14.57 \\
 & 1 & 2.67 & 8.28 & 6.69 & 14.45 \\
\cline{1-6}
\bottomrule
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
 & LNS & MTSP & DOTSP & NNSVF \\
\midrule
0 & 1.80 & 6.72 & 4.72 & 13.41 \\
\bottomrule
\end{tabular}



LNS  MTSP  DOTSP  NNSVF
$n$ serv                          
6   0     0.77  4.41   3.46  13.61
    1     2.17  8.78   5.56  13.15
8   0     0.84  4.57   2.86  12.69
    1     3.00  8.20   5.64  12.01
10  0     1.35  6.07   4.13  14.57
    1     2.67  8.28   6.69  14.45

## Large instances $(n \ge 15)$

In [8]:
res = df[df.n >= 15]

# Get relevant columns and groupby
res = res.groupby(['$n$', '$\omega^T$'])[ALGS].mean().round(2)

print(res[ALGS].to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{llrrrr}
\toprule
 &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & $\omega^T$ &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{15} & 0.5 & 0.16 & 4.76 & 3.47 & 7.68 \\
 & 1.0 & 0.13 & 3.25 & 2.65 & 11.86 \\
 & 2.0 & 0.44 & 2.33 & 1.95 & 19.58 \\
\cline{1-6}
\multirow[t]{3}{*}{20} & 0.5 & 0.16 & 4.70 & 3.74 & 7.87 \\
 & 1.0 & 0.36 & 3.29 & 2.82 & 12.20 \\
 & 2.0 & 0.83 & 2.23 & 2.07 & 20.53 \\
\cline{1-6}
\multirow[t]{3}{*}{25} & 0.5 & 0.15 & 4.05 & 3.54 & 7.62 \\
 & 1.0 & 0.52 & 2.90 & 2.84 & 12.23 \\
 & 2.0 & 0.72 & 2.27 & 1.78 & 20.46 \\
\cline{1-6}
\multirow[t]{3}{*}{30} & 0.5 & 0.11 & 4.47 & 3.52 & 6.54 \\
 & 1.0 & 0.35 & 3.82 & 2.63 & 10.95 \\
 & 2.0 & 1.75 & 2.36 & 1.37 & 19.08 \\
\cline{1-6}
\multirow[t]{3}{*}{35} & 0.5 & 0.31 & 4.00 & 3.06 & 6.61 \\
 & 1.0 & 1.03 & 3.13 & 2.30 & 11.07 \\
 & 2.0 & 2.65 & 2.45 & 1.51 & 19.67 \\
\cline{1-6}
\multirow[t]{3}{*}{40} & 0.5 & 0.28 & 4.50 & 3.28 & 6.33 \\
 & 1.0 & 0.89 & 3.82 & 2.54 & 10.79 \\
 & 2.0 & 2.93 & 2.60 & 1.44 & 19.06 \\
\c

LNS  MTSP  DOTSP  NNSVF
$n$ $\omega^T$                          
15  0.5         0.16  4.76   3.47   7.68
    1.0         0.13  3.25   2.65  11.86
    2.0         0.44  2.33   1.95  19.58
20  0.5         0.16  4.70   3.74   7.87
    1.0         0.36  3.29   2.82  12.20
    2.0         0.83  2.23   2.07  20.53
25  0.5         0.15  4.05   3.54   7.62
    1.0         0.52  2.90   2.84  12.23
    2.0         0.72  2.27   1.78  20.46
30  0.5         0.11  4.47   3.52   6.54
    1.0         0.35  3.82   2.63  10.95
    2.0         1.75  2.36   1.37  19.08
35  0.5         0.31  4.00   3.06   6.61
    1.0         1.03  3.13   2.30  11.07
    2.0         2.65  2.45   1.51  19.67
40  0.5         0.28  4.50   3.28   6.33
    1.0         0.89  3.82   2.54  10.79
    2.0         2.93  2.60   1.44  19.06

In [9]:
res.mean()

LNS       0.765000
MTSP      3.385000
DOTSP     2.583889
NNSVF    12.785000
dtype: float64

In [10]:
res = df[df.n >= 15]

# Get relevant columns and groupby
res = res.groupby(['$n$', 'serv'])[ALGS].mean().round(2)

print(res[ALGS].to_latex(float_format="%.2f", escape=True))
print(res.mean().to_frame().T.to_latex(float_format="%.2f", escape=True))
res

\begin{tabular}{llrrrr}
\toprule
 &  & LNS & MTSP & DOTSP & NNSVF \\
$n$ & serv &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{15} & 0 & 0.34 & 2.69 & 2.19 & 14.90 \\
 & 1 & 0.14 & 4.20 & 3.19 & 11.19 \\
\cline{1-6}
\multirow[t]{2}{*}{20} & 0 & 0.75 & 2.65 & 1.84 & 15.09 \\
 & 1 & 0.15 & 4.16 & 3.91 & 11.98 \\
\cline{1-6}
\multirow[t]{2}{*}{25} & 0 & 0.88 & 1.45 & 1.11 & 14.53 \\
 & 1 & 0.04 & 4.70 & 4.33 & 12.34 \\
\cline{1-6}
\multirow[t]{2}{*}{30} & 0 & 1.47 & 2.09 & 0.92 & 13.78 \\
 & 1 & 0.00 & 5.01 & 4.10 & 10.60 \\
\cline{1-6}
\multirow[t]{2}{*}{35} & 0 & 2.51 & 1.48 & 0.50 & 13.41 \\
 & 1 & 0.15 & 4.90 & 4.08 & 11.48 \\
\cline{1-6}
\multirow[t]{2}{*}{40} & 0 & 2.73 & 1.43 & 0.21 & 12.93 \\
 & 1 & 0.00 & 5.84 & 4.63 & 11.20 \\
\cline{1-6}
\bottomrule
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
 & LNS & MTSP & DOTSP & NNSVF \\
\midrule
0 & 0.76 & 3.38 & 2.58 & 12.79 \\
\bottomrule
\end{tabular}



LNS  MTSP  DOTSP  NNSVF
$n$ serv                          
15  0     0.34  2.69   2.19  14.90
    1     0.14  4.20   3.19  11.19
20  0     0.75  2.65   1.84  15.09
    1     0.15  4.16   3.91  11.98
25  0     0.88  1.45   1.11  14.53
    1     0.04  4.70   4.33  12.34
30  0     1.47  2.09   0.92  13.78
    1     0.00  5.01   4.10  10.60
35  0     2.51  1.48   0.50  13.41
    1     0.15  4.90   4.08  11.48
40  0     2.73  1.43   0.21  12.93
    1     0.00  5.84   4.63  11.20